<a href="https://colab.research.google.com/github/dorian-goueytes/L1_P-M1_MIASHS/blob/main/TD1_HH_neurone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Installation des dépendances
!pip install brian2

In [ ]:
# @title Importation des modules nécessaires
import brian2 as b2
import matplotlib.pyplot as plt
import numpy as np
#import neurodynex3.tools.input_factory as input_factory

In [ ]:
# @title Création d'une fonction pour injecter du courant et pour monitorer l'état de nos neurones
def plot_data(ref, state_monitor, title=None):
    """Plots the state_monitor variables ["vm", "I_e", "m", "n", "h"] vs. time.

    Args:
        state_monitor (StateMonitor): the data to plot
        title (string, optional): plot title to display
    """

    plt.subplot(211)
    plt.plot(ref.t / b2.ms, ref.vm[0] / b2.mV, lw=2, color = 'k', linestyle = '--', alpha = 0.7)
    plt.plot(state_monitor.t / b2.ms, state_monitor.vm[0] / b2.mV, lw=2, alpha = 0.7)

    plt.xlabel("t [ms]")
    plt.ylabel("v [mV]")
    plt.grid()

    #plt.subplot(312)

    #plt.plot(state_monitor.t / b2.ms, state_monitor.m[0] / b2.volt, "black", lw=2)
    #plt.plot(state_monitor.t / b2.ms, state_monitor.n[0] / b2.volt, "blue", lw=2)
    #plt.plot(state_monitor.t / b2.ms, state_monitor.h[0] / b2.volt, "red", lw=2)
    #plt.xlabel("t (ms)")
    #plt.ylabel("act./inact.")
    #plt.legend(("m", "n", "h"))
    #plt.ylim((0, 1))
    plt.grid()

    plt.subplot(212)
    plt.plot(ref.t / b2.ms, ref.I_e[0] / b2.uamp, lw=2, color = 'k', linestyle = '--', alpha = 0.7)
    plt.plot(state_monitor.t / b2.ms, state_monitor.I_e[0] / b2.uamp, lw=2, alpha = 0.7)

    if np.max(ref.t / b2.ms)>np.max(state_monitor.t / b2.ms):
      plt.axis((
          0,
          np.max(ref.t / b2.ms),
          min(ref.I_e[0] / b2.uamp) * 1.1,
          max(ref.I_e[0] / b2.uamp) * 1.1
      ))
    if np.max(state_monitor.t / b2.ms)>=np.max(ref.t / b2.ms):
      plt.axis((
          0,
          np.max(state_monitor.t / b2.ms),
          min(state_monitor.I_e[0] / b2.uamp) * 1.1,
          max(state_monitor.I_e[0] / b2.uamp) * 1.1
      ))

    plt.xlabel("t [ms]")
    plt.ylabel("I [micro A]")
    plt.grid()

    if title is not None:
        plt.suptitle(title)
    plt.tight_layout()
    plt.show()

def get_step_current(t_start, t_end, unit_time, amplitude, append_zero=True):

    """Creates a step current. If t_start == t_end, then a single
    entry in the values array is set to amplitude.

    Args:
        t_start (int): start of the step
        t_end (int): end of the step
        unit_time (Brian2 unit): unit of t_start and t_end. e.g. 0.1*brian2.ms
        amplitude (Quantity): amplitude of the step. e.g. 3.5*brian2.uamp
        append_zero (bool, optional): if true, 0Amp is appended at t_end+1.
        Without that trailing 0, Brian reads out the last value in the array (=amplitude) for all indices > t_end.

    Returns:
        TimedArray: Brian2.TimedArray
    """

    assert isinstance(t_start, int), "t_start_ms must be of type int"
    assert isinstance(t_end, int), "t_end must be of type int"
    assert b2.units.fundamentalunits.have_same_dimensions(amplitude, b2.amp), \
        "amplitude must have the dimension of current e.g. brian2.uamp"
    tmp_size = 1 + t_end  # +1 for t=0
    if append_zero:
        tmp_size += 1
    tmp = np.zeros((tmp_size, 1)) * b2.amp
    tmp[t_start: t_end + 1, 0] = amplitude
    curr = b2.TimedArray(tmp, dt=1. * unit_time)
    return curr

In [ ]:
# @title Création d'un modèle de neurone Hogkin et Huxley
def simulate_HH_neuron(input_current, simulation_time, kcurrent = None):

    """A Hodgkin-Huxley neuron implemented in Brian2.

    Args:
        input_current (TimedArray): Input current injected into the HH neuron
        simulation_time (float): Simulation time [seconds]

    Returns:
        StateMonitor: Brian2 StateMonitor with recorded fields
        ["vm", "I_e", "m", "n", "h"]
    """

    # neuron parameters
    El = 10.6 * b2.mV
    if kcurrent == None:
      EK = -12 * b2.mV
    else:
      EK = kcurrent* b2.mV
    ENa = 115 * b2.mV
    gl = 0.3 * b2.msiemens
    gK = 36 * b2.msiemens
    gNa = 120 * b2.msiemens
    C = 1 * b2.ufarad




    # forming HH model with differential equations
    eqs = """
    I_e = input_current(t,i) : amp
    membrane_Im = I_e + gNa*m**3*h*(ENa-vm) + gl*(El-vm) + gK*n**4*(EK-vm) : amp
    alphah = .07*exp(-.05*vm/mV)/ms    : Hz
    alpham = .1*(25*mV-vm)/(exp(2.5-.1*vm/mV)-1)/mV/ms : Hz
    alphan = .01*(10*mV-vm)/(exp(1-.1*vm/mV)-1)/mV/ms : Hz
    betah = 1./(1+exp(3.-.1*vm/mV))/ms : Hz
    betam = 4*exp(-.0556*vm/mV)/ms : Hz
    betan = .125*exp(-.0125*vm/mV)/ms : Hz
    dh/dt = alphah*(1-h)-betah*h : 1
    dm/dt = alpham*(1-m)-betam*m : 1
    dn/dt = alphan*(1-n)-betan*n : 1
    dvm/dt = membrane_Im/C : volt
    """

    neuron = b2.NeuronGroup(1, eqs, method="exponential_euler")

    # parameter initialization
    neuron.vm = 0

    neuron.m = 0.05
    neuron.h = 0.60
    neuron.n = 0.32

    # tracking parameters
    st_mon = b2.StateMonitor(neuron, ["vm", "I_e", "m", "n", "h"], record=True)
    #st_mon = b2.StateMonitor(neuron, ["vm", "I_e"], record=True)

    # running the simulation
    hh_net = b2.Network(neuron)
    hh_net.add(st_mon)
    hh_net.run(simulation_time)


    return st_mon

### Création d'un neurone de référence
Ce segment permet d'observer le comportement d'un neurone classique pour un courant injecté de 1ms et 7.2mA

Ce neurone nous servira de comparaison pour les manipulations effectuées au cours du TD. Il sera toujours représenté par une courbe noire en pointillés

In [ ]:
current = get_step_current(10, 11, b2.ms, 7.2 * b2.uA)
ref = simulate_HH_neuron(current, 70 * b2.ms,-12)

# @title Création d'un neurone référence

current = get_step_current(10, 11, b2.ms, 7.2 * b2.uA)
state_monitor = simulate_HH_neuron(current, 70 * b2.ms,-12)

plot_data(ref,state_monitor, title="HH Neuron, step current")

### Manipulation de l'intensité du courant d'entrée (partie 1)
L'objectif de cet exercice est de comprendre comment le courant injecté dans un neurone (input) affecte son comportement.
1 paramètres peuvent être ajustés, I_amp


*   I_amp correspond à l'intensité du courant injecté dans notre neurone

**Q1: A quel élément présent dans le système nerveux correspond le courant injecté?**

**Q2: Qu'observez-vous si vous augmentez la valeur de I_amp? Pouvez-vous interprêter ce résultat au regard du cours?**

**Q3: Qu'observez-vous si vous diminuez considérablement I_amp? Quelle est votre interprétation de ce résultats?**

**Q4: A partir de quelle intensité de courant observez-vous une bascule?**





In [ ]:
current = get_step_current(10, 11, b2.ms, 7.2 * b2.uA)
ref = simulate_HH_neuron(current, 70 * b2.ms,-12)

# @title Utilisez les sliders pour manipuler l'intensité du courant injecté
I_amp = 7.2 # @param {type:"slider", min:0, max:24, step:0.1}

#current_pulse_duration = 10 # @param {type:"integer"}
#I_duration = 1 # @param {type:"slider", min:1, max:60, step:1}



current = get_step_current(10, 10+1, b2.ms, I_amp * b2.uA)
state_monitor = simulate_HH_neuron(current, 70 * b2.ms,-12)

plot_data(ref,state_monitor, title="HH Neuron, step current")

### Manipulation de la durée du courant d'entrée (partie 2)
L'objectif de cet exercice est de comprendre comment le courant injecté dans un neurone (input) affect son comportement.
Cette fois-ci 2 paramètres peuvent être ajustés, I_amp et I_duration


*   I_amp correspond toujours à l'intensité du courant injecté dans notre neurone
*   I_duration correspond à la durée du courant injecté

**Q5: Que se passe-t-il lorsque vous augmentez la durée du courant injecté?** (Indice, réfléchissez en termes de nombre de potentiels d'action)

**Q6: Pouvez-vous expliquer cette observation?**

**Q7: Que se passe-t-il si vous augmentez simultanément l'intensité et la durée?**

**Q8: Dans le cas ou nous injectons continuellement du courant, pourquoi notre neurone n'émet-il pas constamment des potentiels d'action?**

**Q9: Comment s'appelle le phénomène empêchant notre neurone d'émettre continuellement des potentiels d'action?**





In [ ]:
current = get_step_current(10, 11, b2.ms, 7.2 * b2.uA)
ref = simulate_HH_neuron(current, 70 * b2.ms,-12)

# @title Utilisez les sliders pour manipuler l'intensité et la durée du courant injecté
I_amp = 7.2 # @param {type:"slider", min:0, max:24, step:0.1}

#current_pulse_duration = 10 # @param {type:"integer"}
I_duration = 1 # @param {type:"slider", min:1, max:55, step:1}



current = get_step_current(10, 10+I_duration, b2.ms, I_amp * b2.uA)
state_monitor = simulate_HH_neuron(current, 70 * b2.ms,-12)

plot_data(ref,state_monitor, title="HH Neuron, step current")

### Manipulation du courant potassique
Cette fois-ci nous allons essayer de comprendre l'effect du courant des ions potassiums sur l'activité de notre neurones.

En plus de I_amp et de I_duration nous ajoutons un 3ème paramètre E_k, correspondant au courant potassique


*   I_amp correspond toujours à l'intensité du courant injecté dans notre neurone
*   I_duration correspond à la durée du courant injecté
*   E_k quantité d'ion K+ pouvant quitter le milieu intracellulaire du neurone

*Conseil: Pour commencer l'exercice laissez I_amp et I_duration sur leur valeur initiales et intéressez-vous uniquement à E_k*

**Q10: Pouvez-vous expliquer cette observation?**

**Q11: Que se passe-t-il si vous augmentez au maximum E_K? Pouvez-vous expliquer ce phénomène?**

**Q12: Décrivez l'effet de place E_k entre la valeur maximale et la valeur de référence. Que se passe-t-il au niveau des potentiels d'action?**

**Q13: Quel phénomène vu dans le cours semble dépendre de E_k?**





In [ ]:
current = get_step_current(10, 60, b2.ms, 7.2 * b2.uA)
ref = simulate_HH_neuron(current, 70 * b2.ms,-12)

# @title Utilisez les sliders pour manipuler l'intensité et la durée du courant injecté
I_amp = 7.2 # @param {type:"slider", min:0, max:24, step:0.1}

#current_pulse_duration = 10 # @param {type:"integer"}
I_duration = 50 # @param {type:"slider", min:1, max:55, step:1}

#current_pulse_duration = 10 # @param {type:"integer"}
E_k = 12 # @param {type:"slider", min:8, max:16, step:1}



current = get_step_current(10, 10+I_duration, b2.ms, I_amp * b2.uA)
state_monitor = simulate_HH_neuron(current, 70 * b2.ms,-E_k)

plot_data(ref,state_monitor, title="HH Neuron, step current")